In [2]:
# -------------------- 
# IMPORTS
# -------------------- 

# ---- LIBRARIES -----
import os
import json
import pandas as pd
import yfinance as yf

from tqdm.auto import tqdm


# -------------------- 
# MAIN
# -------------------- 

class Dataset():

    def __init__(self, mesh):
        self.mesh = mesh
        self.hist = pd.DataFrame([])
        self.date_name = ''

    def load(self):
        
        if self.mesh == '1m':
            self.path = os.getcwd() + '\\resources\\full_NASDAQ_history_1m_Full.csv'
            self.date_name = 'Datetime'
            self.hist = pd.read_csv(self.path, usecols=['Close', 'Company', 'Datetime', 'Dividends', 'High', 'Low', 'Open', 'Stock Splits', 'Volume'])

        else:
            self.path = os.getcwd() + '\\resources\\full_NASDAQ_history.csv'
            self.date_name = 'Date'
            self.hist = pd.read_csv(self.path, usecols=['Close', 'Company', 'Date', 'Dividends', 'High', 'Low', 'Open', 'Stock Splits', 'Volume'])
        
        # Date formating
        return(self.hist)


    def update(self, update_period='max'):
        # Loading Data        
        for company in tqdm(companies_list):
            # Get history Ignore timezone
            hist = yf.Ticker(company).history(period=update_period, interval=self.mesh)

            if hist.empty != True:
                # Reset index, add company name and format date
                hist = hist.reset_index()
                hist['Company'] = [company]*len(hist)

                if self.date_name == '1m':                
                    hist[self.date_name] = hist[self.date_name].dt.strftime('%Y-%m-%d %H:%M:%S')
                else:
                    hist[self.date_name] = hist[self.date_name].astype('datetime64[ns]')

                # Add hist to the list of dict to merge
                list_of_df_to_merge.append(hist)

        # Concat and remove duplicates
        new_hist = pd.concat(list_of_df_to_merge)[hist.columns]
        new_hist = new_hist.drop_duplicates(subset=[self.date_name, 'Company'], keep='last')

        # reset index for the new dataframe
        new_hist = new_hist.reset_index(drop=True)
        new_hist
        return(self.hist)

    def save(self):
        self.hist.to_csv(self.path)


    def new_format(self, study_length):
        #TCD
        return()
            

In [22]:
# Test lines, executed only when the file is executed as main
full_hist = Dataset('1m')
full_hist.load()
full_hist.hist['Datetime'] = full_hist.hist['Datetime'].astype('datetime64[ns]')
full_hist.hist['Datetime'] = pd.to_datetime(full_hist.hist['Datetime'])
full_hist.hist['Datetime'] = full_hist.hist['Datetime'].dt.floor('min')
full_hist.hist

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2020-11-24 09:30:00,113.908997,113.940002,113.908997,113.930000,5102883,0.0,0,AAPL
1,2020-11-24 09:31:00,113.919998,114.029999,113.800003,113.889999,866544,0.0,0,AAPL
2,2020-11-24 09:32:00,113.879997,113.910004,113.710098,113.726601,613673,0.0,0,AAPL
3,2020-11-24 09:33:00,113.730003,113.809998,113.279999,113.300003,934997,0.0,0,AAPL
4,2020-11-24 09:34:00,113.300003,113.309998,112.764999,112.936096,1380426,0.0,0,AAPL
...,...,...,...,...,...,...,...,...,...
146989,2020-12-07 17:59:00,60.654999,60.654999,60.619999,60.619999,8791,0.0,0,GILD
146990,2020-12-07 18:00:00,60.621101,60.639999,60.613800,60.639999,14831,0.0,0,GILD
146991,2020-12-07 18:01:00,60.645000,60.650002,60.639999,60.648800,4791,0.0,0,GILD
146992,2020-12-07 18:02:00,60.650002,60.650002,60.630001,60.650002,6511,0.0,0,GILD


In [4]:
# Loading Data    
date_name = 'Datetime'
companies_list = pd.read_csv(os.getcwd() + '\\resources\\NASDAQ_Companies_List.csv')['Companies'].to_list()

list_of_df_to_merge=[full_hist.hist]

for company in tqdm(companies_list):
    # Get history Ignore timezone
    hist = yf.Ticker(company).history(period='2d', interval='1m')

    if hist.empty != True:
        # Reset index, add company name and format date
        hist = hist.reset_index()
        hist['Company'] = [company]*len(hist)

        if '1m' == '1m':                
            hist[date_name] = hist[date_name].dt.strftime('%Y-%m-%d %H:%M:%S')
            hist[date_name] = pd.to_datetime(hist[date_name])
            
        else:
            hist[date_name] = hist[date_name].astype('datetime64[ns]')
            hist[date_name] = pd.to_datetime(hist[date_name])

        # Add hist to the list of dict to merge
        list_of_df_to_merge.append(hist)

# Concat and remove duplicates
new_hist = pd.concat(list_of_df_to_merge)[hist.columns]
new_hist = new_hist.drop_duplicates(subset=[date_name, 'Company'], keep='last')

# reset index for the new dataframe
new_hist = new_hist.reset_index(drop=True)
new_hist

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2020-11-24 09:30:00,113.908997,113.940002,113.908997,113.930000,5102883,0.0,0,AAPL
1,2020-11-24 09:31:00,113.919998,114.029999,113.800003,113.889999,866544,0.0,0,AAPL
2,2020-11-24 09:32:00,113.879997,113.910004,113.710098,113.726601,613673,0.0,0,AAPL
3,2020-11-24 09:33:00,113.730003,113.809998,113.279999,113.300003,934997,0.0,0,AAPL
4,2020-11-24 09:34:00,113.300003,113.309998,112.764999,112.936096,1380426,0.0,0,AAPL
...,...,...,...,...,...,...,...,...,...
169569,2020-12-09 15:55:00,60.830002,60.950001,60.825001,60.930000,42309,0.0,0,GILD
169570,2020-12-09 15:56:00,60.930000,60.965000,60.910000,60.955002,41748,0.0,0,GILD
169571,2020-12-09 15:57:00,60.939999,60.970001,60.919998,60.945000,38139,0.0,0,GILD
169572,2020-12-09 15:58:00,60.950001,60.959999,60.919998,60.939999,44516,0.0,0,GILD


In [11]:
import numpy as np

TCD = pd.pivot_table(new_hist, 'Open', index=['Company'], columns=['Datetime'], aggfunc=np.sum, margins=True, margins_name='NASDAQ').fillna(method='ffill', axis=1)

TCD = TCD.drop(columns=['NASDAQ'])
TCD = TCD.reindex(TCD.columns.tolist().sort(), axis=1)

TCD

Datetime,2020-11-24 09:30:00,2020-11-24 09:31:00,2020-11-24 09:32:00,2020-11-24 09:33:00,2020-11-24 09:34:00,2020-11-24 09:35:00,2020-11-24 09:36:00,2020-11-24 09:37:00,2020-11-24 09:38:00,2020-11-24 09:39:00,...,2020-12-09 15:50:00,2020-12-09 15:51:00,2020-12-09 15:52:00,2020-12-09 15:53:00,2020-12-09 15:54:00,2020-12-09 15:55:00,2020-12-09 15:56:00,2020-12-09 15:57:00,2020-12-09 15:58:00,2020-12-09 15:59:00
Company,,,,,,,,,,,,,,,,,,,,,
AAPL,113.908997,113.919998,113.879997,113.730003,113.300003,112.933296,113.125000,113.250000,113.175003,112.980003,...,121.699997,121.834999,121.779999,121.629997,121.764999,121.779999,121.919998,121.885002,121.989998,121.959999
AMAT,81.000000,81.059998,80.900002,80.855003,80.879997,80.550003,80.550003,80.175003,79.959999,79.970001,...,87.809998,87.769997,87.790001,87.769997,87.875000,87.834999,87.949997,87.885002,87.910004,87.910004
AMRN,4.450000,4.470000,4.450000,4.440100,4.470000,4.480000,4.465000,4.460000,4.490000,4.500000,...,4.620000,4.620000,4.600000,4.600000,4.600000,4.600000,4.625000,4.595000,4.595000,4.590000
ATVI,75.930000,75.885002,75.735001,75.690002,75.940002,76.014999,75.430000,75.495003,75.480003,75.400002,...,82.379997,82.330002,82.364998,82.330002,82.394997,82.629997,82.709999,82.669998,82.660004,82.800003
BLDP,20.240000,20.780001,20.549999,20.980000,21.200001,20.879999,20.940001,20.753700,20.580000,20.650000,...,19.680000,19.695000,19.719999,19.690001,19.700001,19.709999,19.715000,19.700001,19.695000,19.700001
CMCSA,50.000000,50.169998,50.200001,50.320000,50.419998,50.349998,50.529999,50.400002,50.349998,50.320000,...,51.305000,51.345001,51.355000,51.314999,51.384998,51.349998,51.389999,51.369999,51.410000,51.410000
CSCO,42.139999,42.060001,42.035000,41.970001,42.025002,42.025002,41.980000,41.939999,42.040001,42.084999,...,44.755001,44.724998,44.724998,44.680000,44.720001,44.695000,44.735001,44.689999,44.715000,44.705002
EBAY,49.919998,49.945000,49.900002,49.880001,49.939999,49.939999,49.970001,49.860001,49.856701,49.889999,...,49.410000,49.424999,49.465000,49.430000,49.480000,49.480000,49.555000,49.564999,49.605000,49.610001
ERIC,12.060000,12.085000,12.085000,12.080000,12.070000,12.069900,12.065000,12.065000,12.075000,12.060000,...,12.320000,12.325000,12.330000,12.330000,12.330000,12.330000,12.330000,12.330000,12.330000,12.335000


In [6]:
#TCD[TCD.columns[x]].fillna(TCD[TCD.columns[x-1]])

TCD[TCD.columns[x]] = TCD.apply(
    lambda row: row[TCD.columns[x-1]] if np.isnan(row[TCD.columns[x]]) else row[TCD.columns[x]]
)

NameError: name 'x' is not defined

In [90]:
TCD = TCD.reset_index().rename_axis(None, axis=1).set_index('Company')
TCD[TCD.columns[:10]]

,2020-11-24 09:30:00,2020-11-24 09:31:00,2020-11-24 09:32:00,2020-11-24 09:33:00,2020-11-24 09:34:00,2020-11-24 09:35:00,2020-11-24 09:36:00,2020-11-24 09:37:00,2020-11-24 09:38:00,2020-11-24 09:39:00
Company,,,,,,,,,,
AAPL,113.908997,113.919998,113.879997,113.730003,113.300003,112.933296,113.125000,113.250000,113.175003,112.980003
AMAT,81.000000,81.059998,80.900002,80.855003,80.879997,80.550003,80.550003,80.175003,79.959999,79.970001
AMRN,4.450000,4.470000,4.450000,4.440100,4.470000,4.480000,4.465000,4.460000,4.490000,4.500000
ATVI,75.930000,75.885002,75.735001,75.690002,75.940002,76.014999,75.430000,75.495003,75.480003,75.400002
BLDP,20.240000,20.780001,20.549999,20.980000,21.200001,20.879999,20.940001,20.753700,20.580000,20.650000
CMCSA,50.000000,50.169998,50.200001,50.320000,50.419998,50.349998,50.529999,50.400002,50.349998,50.320000
CSCO,42.139999,42.060001,42.035000,41.970001,42.025002,42.025002,41.980000,41.939999,42.040001,42.084999
EBAY,49.919998,49.945000,49.900002,49.880001,49.939999,49.939999,49.970001,49.860001,49.856701,49.889999
ERIC,12.060000,12.085000,12.085000,12.080000,12.070000,12.069900,12.065000,12.065000,12.075000,12.060000


In [91]:
list = []
import math
table = TCD.reset_index().rename_axis(None, axis=1).set_index('Company')

for x in range(len(TCD.columns)):
    new_list = table[TCD.columns[x]].to_list()
    count = sum(math.isnan(x) for x in new_list)
    
    list.append(count)
        
print(list)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 2, 2, 1, 1, 1, 0, 0, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 0, 2, 2, 1, 2, 1, 2, 3, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 3, 30, 1, 3, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 0, 1, 3, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 1, 2, 1, 0, 1, 2, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 3, 1, 2, 1, 1, 3, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 3, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 3, 2, 2, 0, 0, 1, 1, 2, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,

In [94]:
table[TCD.columns[2]].to_list()

[113.87999725341795,
 80.9000015258789,
 4.449999809265137,
 75.73500061035156,
 20.549999237060547,
 50.20000076293945,
 42.03499984741211,
 49.9000015258789,
 12.085000038146973,
 10.717000007629396,
 16.75,
 59.9000015258789,
 9.579999923706056,
 0.13910000026226044,
 6.630000114440918,
 40.25,
 12.550000190734865,
 5.699999809265138,
 46.334999084472656,
 16.586200714111328,
 99.18499755859376,
 44.47999954223633,
 210.25,
 524.3250122070311,
 7.1599998474121085,
 28.23200035095215,
 143.5,
 97.6999969482422,
 6.6500000953674325,
 3.994999885559082,
 nan]

In [93]:
L = []
for x in new_hist['Datetime']:
    if not(type(x) in L):
        print(type(x))
        L.append(type(x))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
type(new_hist['Datetime'][0])

In [ ]:
hist['Datetime'] = pd.to_datetime(hist['Datetime'])

In [ ]:
TCD.columns.name = None   

In [ ]:
TCD.reset_index()

In [114]:
TCD[TCD.columns[:10]]

,2020-11-24 09:30:00,2020-11-24 09:31:00,2020-11-24 09:32:00,2020-11-24 09:33:00,2020-11-24 09:34:00,2020-11-24 09:35:00,2020-11-24 09:36:00,2020-11-24 09:37:00,2020-11-24 09:38:00,2020-11-24 09:39:00
Company,,,,,,,,,,
AAPL,113.908997,113.919998,113.879997,113.730003,113.300003,112.933296,113.125000,113.250000,113.175003,112.980003
AMAT,81.000000,81.059998,80.900002,80.855003,80.879997,80.550003,80.550003,80.175003,79.959999,79.970001
AMRN,4.450000,4.470000,4.450000,4.440100,4.470000,4.480000,4.465000,4.460000,4.490000,4.500000
ATVI,75.930000,75.885002,75.735001,75.690002,75.940002,76.014999,75.430000,75.495003,75.480003,75.400002
BLDP,20.240000,20.780001,20.549999,20.980000,21.200001,20.879999,20.940001,20.753700,20.580000,20.650000
CMCSA,50.000000,50.169998,50.200001,50.320000,50.419998,50.349998,50.529999,50.400002,50.349998,50.320000
CSCO,42.139999,42.060001,42.035000,41.970001,42.025002,42.025002,41.980000,41.939999,42.040001,42.084999
EBAY,49.919998,49.945000,49.900002,49.880001,49.939999,49.939999,49.970001,49.860001,49.856701,49.889999
ERIC,12.060000,12.085000,12.085000,12.080000,12.070000,12.069900,12.065000,12.065000,12.075000,12.060000


In [104]:
to_deduplicate = full_hist.hist['Company'].to_list()

In [107]:
L = []
for x in to_deduplicate:
    if x not in L:
        L.append(x)

In [113]:
import csv
L

with open(os.getcwd() + '\\resources\\full_NASDAQ_history_1m_Full.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(L)

In [23]:
import json
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}], timeout=30, max_retries=10, retry_on_timeout=True)
companies_list = pd.read_csv(os.getcwd() + '\\resources\\NASDAQ_Companies_List.csv')['Companies'].to_list()

with open(os.getcwd() + '\\resources\\hist_json_1d.json') as json_file:
    es_dict = json.load(json_file)

for company in tqdm(companies_list):
    res = es.index(index='hist_1d', id=company, body=es_dict[company])

print(res['result'])

es.indices.refresh(index="inbox")

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x00000219AF95D490>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x00000219AF95D490>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it)

In [3]:
test = yf.Ticker('MSFT')

In [22]:
test.recommendations

,Firm,To Grade,From Grade,Action
Date,,,,
2012-03-16 08:19:00,Argus Research,Buy,,up
2012-03-19 14:00:00,Hilliard Lyons,Long-Term Buy,,main
2012-03-22 07:03:00,Morgan Stanley,Overweight,,main
2012-04-03 11:53:00,UBS,Buy,,main
2012-04-20 06:18:00,Deutsche Bank,Buy,,main
...,...,...,...,...
2020-10-28 12:00:17,Morgan Stanley,Overweight,,main
2020-10-28 13:22:56,Credit Suisse,Outperform,,main
2020-10-29 16:00:20,Deutsche Bank,Buy,,main


In [23]:
list(set(test.recommendations['To Grade'].to_list()))

['Hold',
 '',
 'Long-Term Buy',
 'Underperform',
 'Sell',
 'Underweight',
 'Sector Perform',
 'Market Perform',
 'Neutral',
 'Overweight',
 'Strong Buy',
 'Buy',
 'Outperform',
 'Equal-Weight',
 'Perform']

In [4]:
test.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 163000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and de

In [1]:
import random

In [4]:
res = {
    '-1':0,
    '0':0,
    '1':0,
}


for i in range(10000):
    res[str(random.choice([-1, 0, 1]))] += 1

print(res)

{'-1': 3285, '0': 3372, '1': 3343}


In [5]:
a = pd.DataFrame([[1,2], [3, 4]], index=['a','b'], columns=['c','d'])

In [10]:
a.loc['a'].to_dict()

{'c': 1, 'd': 2}